# Template

## Primes

In [10]:
import math
def gcd(a, b):
    return b if a == 0 else gcd(b, a%b)
# O(NlogN)
def eratosthenes_sieve(N):
    ans = []
    flags = [0] * (N+1)
    sqrtn = int(math.sqrt(N))
    for i in range(2, sqrtn+1):
        if not flags[i]:
            ans.append(i)
            for j in range(i*i,N+1,i):
                flags[j] = 1
    for i in range(sqrtn, N+1):
        if not flags[i]: ans.append(i)
    return ans
# 每个合数只被筛一次 O(N)
def euler_sieve(N):
    ans = []
    flags = [0] * (N+1)
    for i in range(2, N+1):
        if not flags[i]: ans.append(i)
        j = 0
        while i*ans[j] <= N:
            flags[i*ans[j]] = 1
            if i%ans[j] == 0: break
            j += 1
    return ans
# N = 1000000
# euler_sieve(N)
# eratosthenes_sieve(N)

## Bisect

In [ ]:
# 记住bisect_left和bisect_right的区别
# 327 (可以用线段树)
class Solution:
    def countRangeSum(self, nums: List[int], lower: int, upper: int) -> int:
        sorts = [0]
        ret = 0
        for s in itertools.accumulate(nums):
            left = bisect.bisect_left(sorts, s-upper)
            right = bisect.bisect_right(sorts, s-lower)
            ret += right - left
            bisect.insort(sorts, s)
        return ret

## Coins

In [ ]:
# 322
class Solution(object):
    def coinChange(self, coins, amount):
        dp = [1<<31]*(amount+1)
        dp[0] = 0
        for i in range(1, amount+1):
            for c in coins:
                if i-c >= 0:
                    dp[i] = min(dp[i], dp[i-c]+1)
        return -1 if dp[amount]==(1<<31) else dp[amount]

## UnionFind

In [ ]:
class UnionFind(object):
    def __init__(self, n):
        self.n = n
        self.parent = list(range(n))
        self.size = [1] * n
    def find(self, idx):
        while self.parent[idx]!=idx:
            self.parent[idx] = self.parent[self.parent[idx]]
            idx = self.parent[idx]
        return idx
    def connect(self, a, b):
        fa, fb = self.find(a), self.find(b)
        if fa!=fb:
            if self.size[fa] > self.size[fb]:
                self.parent[fb] = fa
                self.size[fa] += self.size[fb]
            else:
                self.parent[fa] = fb
                self.size[fb] += self.size[fa]

## ShortestPath

In [ ]:
# 贪心 BFS
def BFS(edges, start, end, N):
    inf = 1<<30
    vis = [inf] * N
    vis[start] = 0
    Q = collections.deque([start])
    while Q:
        cur = Q.popleft()
        if cur == end: return vis[cur]
        for nxt, w in edges[cur]:
            if vis[nxt] > vis[cur] + w:
                vis[nxt] = vis[cur] + w
                Q.append(nxt)      
# 比2时间复杂度高，堆中会加入所有边
def Dijkstra_1(edges, start, end, N):
    inf = 1<<30
    vis = [0] * N
    Q = [(0, start)]
    while Q:
        cost, cur = heapq.heappop(Q)
        if vis[cur]: continue
        vis[cur] = 1
        if cur == end: return cost
        for nxt, w in edges[cur]:
            if not vis[nxt]:
                heapq.heappush(Q, (cost+w, nxt))               
# O(ElogV) 适合正权稀疏图，用邻接链表存储
def Dijkstra_2(edges, start, end, N): # IMPORTANT
    inf = 1<<30
    vis = [0] * N
    dist = [inf] * N
    dist[start] = 0
    Q = [(0, start)]
    while Q:
        cost, cur = heapq.heappop(Q)
        if vis[cur]: continue
        vis[cur] = 1
        if cur == end: return cost
        for nxt, w in edges[cur]:
            if dist[nxt] > dist[cur] + w:
                dist[nxt] = dist[cur] + w
                heapq.heappush(Q, (dist[nxt], nxt))
# O(V^2) 适合稠密图
def Dijkstra(edges, start, end, N):
    inf = 1<<30
    vis = [0] * N
    dist = [inf] * N
    dist[start] = 0
    for i in range(N):
        v, minw = -1, inf
        for j in range(N):
            if not vis[j] and dist[j]<=minw:
                v, minw = j, dist[j]
        vis[v] = 1
        for j in range(N):
            dist[j] = min(dist[j], dist[v]+edges[v][j])
        if v == end: return dist[end]
# O(V^3)
def Floyd(edges, N):
    inf = 1<<30
    dist = [[inf]*N for _ in range(N)]
    for i in range(N):
        dist[i][i] = 0
    for k in range(N):
        for i in range(N):
            for j in range(N):
                dist[i][j] = min(dist[i][j], dist[i][k]+dist[k][j])

## Graph2Tree

In [ ]:
# Prim适合稠密图，Krustral适合稀疏图
class Graph(object):
    def __init__(self, maps):
        self.maps = maps
        self.nodenum = self.get_nodenum()
        self.edgenum = self.get_edgenum()
    def kruskal(self): # 加边法
        if self.nodenum <= 0 or self.edgenum < self.nodenum-1:
            return 0
        ans = 0
        edge_list = []
        for i in range(self.nodenum):
            for j in range(i,self.nodenum):
                if self.maps[i][j] < 9999:
                    edge_list.append([i, j, self.maps[i][j]])
        edge_list.sort(key=lambda a:a[2])
        edge_num = 0
        uf = UnionFind(self.nodenum)
        for a, b, w in edge_list:
            if uf.find(a)!=uf.find(b):
                uf.connect(a, b)
                ans += w
                edge_num += 1
            if edge_num == self.nodenum - 1: return ans
        return -1
    # 优先队列实现 O(E*logV)
    def prim(self): # 加点法
        if self.nodenum <= 0 or self.edgenum < self.nodenum-1:
            return 0
        ans = 0
        selected_node = [0]
        candidate_node = list(range(1, self.nodenum))
        Q = [(self.maps[j][0], j) for j in candidate_node]
        heapq.heapify(Q)
        
        while candidate_node:
            while Q and Q[0][1] in selected_node:
                heapq.heappop(Q)
            min_dist, pt = Q[0]
            heapq.heappop(Q)
            
            ans += min_dist
            selected_node.append(pt)
            candidate_node.remove(pt)
            for next_pt in candidate_node:
                heapq.heappush(Q, (self.maps[pt][next_pt], next_pt))
        return ans
    # O(V^2)实现
    def prim(self):
        if self.nodenum <= 0 or self.edgenum < self.nodenum-1:
            return 0
        ans = 0
        inf = 1<<31
        vis = [0] * self.nodenum
        min_dist = [inf] * self.nodenum
        min_dist[0] = 0
        for i in range(self.nodenum):
            u, gmin = -1, inf
            for j in range(self.nodenum):
                if not vis[j] and min_dist[j] <= gmin:
                    gmin = min_dist[j]
                    u = j
            ans += gmin
            vis[u] = 1
            for j in range(self.nodenum):
                if not vis[j]:
                    min_dist[j] = min(min_dist[j], self.maps[u][j])
        return ans

## MaxArea

In [ ]:
# 85
class Solution(object):
    def maximalRectangle(self, matrix): 
        if len(matrix) == 0 or len(matrix[0]) == 0: return 0
        M, N = len(matrix), len(matrix[0])
        H = [0] * (N+1)
        ans = 0
        for i, row in enumerate(matrix):
            stack = []
            hs = row + ["0"]
            for j, val in enumerate(hs):
                if val == "1": H[j] += 1
                else: H[j] = 0
                while stack and H[j] < H[stack[-1]]:
                    s = stack.pop(-1)
                    ans = max(ans, H[s]*(j-stack[-1]-1 if stack else j))
                stack.append(j)
        return ans
# 221
class Solution(object):
    def maximalSquare(self, mat):
        if len(mat) == 0 or len(mat[0]) == 0: return 0
        M, N = len(mat), len(mat[0])
        dp = [[0]*(N+1) for _ in range(M+1)]
        ans = 0
        for i in range(M):
            for j in range(N):
                if mat[i][j] == '0': continue
                dp[i+1][j+1] = min(dp[i][j], dp[i][j+1], dp[i+1][j]) + 1
                ans = max(ans, dp[i+1][j+1])
        return ans * ans

## SegmentTree

In [ ]:
# pos, left, right 一般为 0,0,N-1
# 区间更新 区间查询
class SegmentTree(object):
    def __init__(self, n):
        self.tree = [0]*4*n
        self.lazy = [0]*4*n
        self.mod = 10**9+7
    # 根据数组建树
    def build_tree(self, pos, left, right, nums):
        if left == right:
            self.tree[pos] = nums[left]
            return
        mid = (left+right)>>1
        self.build_tree(2*pos+1, left, mid, nums)
        self.build_tree(2*pos+2, mid+1, right, nums)
        self.tree[pos] = self.tree[2*pos+1]+self.tree[2*pos+2]
    #单点更新
    def update(self, pos, left, right, index, val):
        if left == right and left == index:
            self.tree[pos] = val
            return
        mid = (left+right)>>1
        if mid >= index: self.update(2*pos+1, left, mid, index, val)
        else: self.update(2*pos+2, mid+1, right, index, val)
        self.tree[pos] = self.tree[2*pos+1]+self.tree[2*pos+2]
    # 区间更新
    def update(self, pos, left, right, curLeft, curRight, nums):
        if right < curLeft or left > curRight: return
        if left == right and curLeft<=left<=curRight:
            self.tree[pos] = nums[left]
            return
        mid = (left+right)>>1
        self.update(2*pos+1, left, mid, curLeft, curRight, val)
        self.update(2*pos+2, mid+1, right, curLeft, curRight, val)
        self.tree[pos] = (self.tree[2*pos+1] + self.tree[2*pos+2]) % self.mod
    # 区间查询
    def query(self, pos, left, right, curLeft, curRight):
        if right < curLeft or left > curRight: return 0
        if left >= curLeft and right <= curRight:
            return self.tree[pos]
        mid = (left+right)>>1
        return (self.query(2*pos+1, left, mid, curLeft, curRight) + self.query(2*pos+2, mid+1, right, curLeft, curRight))%self.mod
    # 区间lazy优化
    def push_down(self, pos, left, right):
        if self.lazy[pos] == 0: return
        self.tree[pos] += self.lazy[pos] * (right-left+1)
        if left < right:
            self.lazy[2*pos+1] += self.lazy[pos]
            self.lazy[2*pos+2] += self.lazy[pos]
        self.lazy[pos] = 0  
    def update(self, pos, left, right, curLeft, curRight, val):
        self.push_down(pos, left, right)
        if right < curLeft or left > curRight: return
        if left>=curLeft and right<=curRight:
            self.lazy[pos] = val
            self.push_down(pos, left, right)
            return
        mid = (left+right)>>1
        self.update(2*pos+1, left, mid, curLeft, curRight, val)
        self.update(2*pos+2, mid+1, right, curLeft, curRight, val)
        self.tree[pos] = (self.tree[2*pos+1] + self.tree[2*pos+2]) % self.mod
    def query(self, pos, left, right, curLeft, curRight):
        if right < curLeft or left > curRight: return 0
        self.push_down(pos, left, right)
        if left>=curLeft and right<=curRight:
            return self.tree[pos]
        mid = (left+right)>>1
        return (self.query(2*pos+1, left, mid, curLeft, curRight) + self.query(2*pos+2, mid+1, right, curLeft, curRight))%self.mod

## Optimization

In [ ]:
# SGD
# 1515

# Problems

In [3]:
import sys
FILE = 0
inp = sys.stdin if not FILE else open('input.txt', 'r')
N,M,K = list(map(int, inp.readline().strip().split(' ')))
arr = list(map(int, inp.readline().strip().split(' ')))

min_val = 1<<31
ans = 0
stack = []
for i, val in enumerate(arr):
    while stack and stack[-1] > val: stack.pop(-1)
    stack.append(val)
    if i>=M and arr[i-M] == stack[0]: stack = stack[1:]
    min_val = stack[0]
    if i >= M-1: ans += int(min_val>=K)
print(ans)

5


In [20]:
import sys
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
N, K, D = list(map(int, inp.readline().strip().split(' ')))
ans, mod = 0, 998244353
dp = [[[0,0] for i in range(N+1)] for _ in range(N+1)]
for cur in range(1, N+1):
    if cur <= K:
        dp[1][cur] = [1, 0] if cur < D else [0, 1]
for n in range(2, N+1):
    for cur in range(1, K+1):
        for zsum in range(n-1, N+1-cur):
            if cur < D: 
                dp[n][cur+zsum][0] += dp[n-1][zsum][0]
                dp[n][cur+zsum][1] += dp[n-1][zsum][1]
            else: 
                dp[n][cur+zsum][1] += sum(dp[n-1][zsum])
            dp[n][cur+zsum][0] %= mod
            dp[n][cur+zsum][1] %= mod
#             print(n, cur, zsum, 'dp[%d][%d]'%(n,cur+zsum), dp[n][cur+zsum])
for n in range(1, N+1):
    ans = (ans + dp[n][N][1]) % mod
print(ans)

12


In [32]:
import sys, heapq
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
inf = 1<<31
T = int(inp.readline().strip())
for epoch in range(T):
    N, M, K = map(int, inp.readline().strip().split(' '))
    edges = [[inf]*(N+1) for _ in range(N+1)]
    for _ in range(M):
        a, b, c = map(int, inp.readline().strip().split(' '))
        edges[a][b] = edges[b][a] = c
    arr, node_id, dd = [], -1, dict()
    for i in range(1, N+1):
        for j in range(i+1, N+1):
            if edges[i][j] <= K:
                arr.append((i, j))
                dd[i] = dd.get(i, []) + [j]
                dd[j] = dd.get(j, []) + [i]
    res = set([1])
    def foo(i):
        if i not in dd:
            return
        for j in dd[i]:
            if j not in res:
                res.add(j)
                foo(j)
    foo(1)
    if len(res) < N: print('No')
    else: print('Yes')

Yes
No


In [53]:
import sys, collections
import datetime as dt
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
inf = 1<<31
N, M = map(int, inp.readline().strip().split(' '))
edges = collections.defaultdict(dict)
for _ in range(M):
    u, v, t = map(int, inp.readline().strip().split(' '))
    edges[u][v] = edges[v][u] = t
start, end, cur_time = inp.readline().strip().split(' ')
start, end = int(start), int(end)
def shortest_path(edges, start, end):
    Q = [(0, start)]
    vis = set()
    while 1:
        cost, cur = heapq.heappop(Q)
        if cur in vis: continue
        vis.add(cur)
        if cur == end: return cost
        for nxt in edges[cur]:
            heapq.heappush(Q, (cost+edges[cur][nxt], nxt))
ans = shortest_path(edges, start, end)
the_time = dt.datetime.strptime(cur_time, '%m.%d/%H')
new_time = the_time + dt.timedelta(hours=ans)
print(str(new_time.month)+'.'+str(new_time.day)+'/'+str(new_time.hour))

7.11/0


In [7]:
import sys, collections
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
N = int(inp.readline().strip())
mat = []
for _ in range(N):
    mat.append(list(inp.readline().strip()))
if N > 2:
    F = dict()
    def father(x):
        if x not in F: 
            F[x] = x
            return x
        if x!=F[x]:
            F[x] = father(F[x])
        return F[x]
    for i in range(1, N-1):
        for j in range(1, N-1):
            val = mat[i][j]
            if val == '1': continue
            for di, dj in [(i,j+1),(i+1,j)]:
                if di<N-1 and dj<N-1 and mat[di][dj] == '0':
                    F[father((di,dj))] = father((i, j))
    for j in [0, N-1]:
        for i in range(N):
            if mat[i][j] == '1': continue
            for di, dj in [(i,j+1),(i+1,j)]:
                if di<N and dj<N and mat[di][dj] == '0':
                    F[father((di,dj))] = father((i, j))
    for i in [0, N-1]:
        for j in range(1, N-1):
            if mat[i][j] == '1': continue
            for di, dj in [(i,j+1),(i+1,j)]:
                if di<N and dj<N and mat[di][dj] == '0':
                    F[father((di,dj))] = father((i, j))
    for i in range(1, N-1):
        for j in range(1, N-1):
            if mat[i][j] == '0':
                fi, fj = father((i, j))
                if 0<fi<N-1 and 0<fj<N-1: mat[i][j] = '1'
print('\n'.join([''.join(line) for line in mat]))

010
111
010


In [13]:
import sys, collections
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
N = int(inp.readline().strip())
mat = []
for _ in range(N):
    mat.append(list(inp.readline().strip()))
if N > 2:
    vis = set()
    def bfs(x, y):
        path = []
        Q = collections.deque([(x, y)])
        vis.add((x, y))
        flag = 0
        while Q:
            i, j = Q.popleft()
            if i == 0 or i == N-1 or j==0 or j == N-1: flag = 1
            path.append((i, j))
            for di, dj in [(i,j+1),(i+1, j),(i-1,j),(i,j-1)]:
                if 0<=di<N and 0<=dj<N and mat[di][dj]=='0' and (di,dj) not in vis:
                        Q.append((di, dj))
                        vis.add((di, dj))
        if not flag:
            for i, j in path:
                mat[i][j] = '1'
    for i in range(N):
        for j in range(N):
            if mat[i][j] == '0' and (i, j) not in vis:
                bfs(i, j)
print('\n'.join([''.join(line) for line in mat]))

1111
0111
1111
0010


In [3]:
import sys, collections
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
M, N = map(int, inp.readline().strip().split(' '))
mat = []
for i in range(M):
    mat.append(map(int, inp.readline().strip().split(' ')))
if M == 0 or N == 0: print(0)
else:
    dp = [[0]*N for _ in range(M)]
    dp[-1][-1] = mat[-1][-1]
    for i in range(M-1, -1, -1):
        for j in range(N-1, -1, -1):
            if i == M-1 and j == N-1: dp[i][j] = mat[i][j]
            elif i == M-1: dp[i][j] = mat[i][j] + dp[i][j+1]
            elif j == N-1: dp[i][j] = mat[i][j] + dp[i+1][j]
            else: dp[i][j] = mat[i][j] + max(dp[i+1][j], dp[i][j+1])
    print(dp[0][0])

23


In [ ]:
# 小米第二题NMS
import sys, collections
from rtree import index
idx = index.Index()
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
N, thres = inp.readline().strip().split(' ')
N, thres = int(N), float(thres)
arr = []
raw_arr = []
for _ in range(N):
    line = inp.readline().strip()
    raw_arr.append(line)
    x1, y1, x2, y2, sc = map(float, line.split(' '))
    arr.append([x1, y1, x2, y2, (x2-x1)*(y2-y1), score])
    if arr[-1][-2]>0.: idx.insert(_, (x1, y1, x2, y2))
def get_score(rec1, rec2):
    x1, y1, x2, y2, area1, s1 = rec1
    a1, b1, a2, b2, area2, s2 = rec2
    if x1>=a2 or y1>=b2 or x2<=a1 or y2<=b1: return 0.
    minx, miny = max(x1, a1), max(y1, b1)
    maxx, maxy = min(x2, a2), min(y2, b2)
    inter = (maxx-minx)*(maxy-miny)
    return inter*1./(area1+area2-inter)
failes = set()
for i in range(N):
    if arr[i][-2] > 0.:
        rec0, s0 = tuple(arr[i][:4]), arr[-1]
        for xid, rec in clf.intersection(rec0):
            area1, s1 = arr[xid][-2:]
            t0 = get_score(i, xid)
            if t0 < thres: continue
            if s0 < s1: failes.add(i)
            else: failes.add(xid)
ans = []
for i in range(N):
    if i not in failes and arr[i][-2]>0.:
        ans.append(raw_arr[i])
print('\n'.join(ans))

In [ ]:
import sys, collections
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
start, end = map(int, inp.readline().strip().split(' '))
N = int(inp.readline().strip().split(' '))
edges = [[0]*101 for _ in range(101)]
edg = [[] for _ in range(101)]
for _ in range(N):
    a, b = map(int, inp.readline().strip().split(' '))
    if not edges[a][b] and not edges[b][a]:
        edges[a][b] = edges[b][a] = 1
        edges[a].append(b)
        edges[b].append(a)
for i in range(101):
    edges[i].sort()
def bfs(s):
    vis = [0] * 101
    Q = collections.deque([s])
    vis[s] = 1
    ans = [s]
    while Q:
        cur = Q.popleft()
        for nxt in edges[cur]:
            if not vis[nxt]:
                ans.append(nxt)
                Q.append(nxt)
                vis[nxt] = 1
    return ans
arr = dict()
for i in range(101):
    if len(edges[i]) == 0: continue
    arr[i] = bfs(i)


In [1]:
import sys, collections
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
N, P = map(int, inp.readline().strip().split(' '))
arr = []
# num, money, power
for _ in range(N):
    arr.append(map(int, inp.readline().strip().split(' ')))
dp = [0] * (P+10)
for i in range(N):
    money, power = arr[i][1], arr[i][2]
    for j in range(money, P+1):
        dp[j] = max(dp[j], dp[j-money]+power)
print(dp[P])

27


In [3]:
import sys, collections
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
T = int(inp.readline().strip())
for t in range(T):
    M, N = map(int, inp.readline().strip().split(' '))
    mat = []
    for _ in range(M):
        mat.append(list(inp.readline().strip()))
    if M == 0 or N == 0: print('NO')
    else:
        start, end = [-1, -1], [-1, -1]
        for i in range(M):
            for j in range(N):
                if mat[i][j] == 'S': start = (i, j)
                elif mat[i][j] == 'E': end = (i, j)
        if start[0] == -1 or end[0] == -1: print('NO')
        else:
            def bfs():
                Q = collections.deque([start])
                ans = 'NO'
                while Q:
                    x, y = Q.popleft()
                    if mat[x][y] == 'E':
                        ans = 'YES'
                        break
                    mat[x][y] = '#'
                    for dx, dy in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]:
                        if 0<=dx<M and 0<=dy<N and mat[dx][dy]!='#':
                            Q.append((dx, dy))
                return ans
            print(bfs())

YES
NO


In [3]:
import sys, collections
FILE = 0
inp = sys.stdin if not FILE else open('input.txt', 'r')
def isMatch(s, p):
    m, n = len(s), len(p)
    dp = [[False]*(n+1) for _ in range(m+1)]
    dp[0][0] = True
    for i in range(1, n+1):
        dp[0][i] = dp[0][i-1] and p[i-1] == '*'
    for i in range(1, m+1):
        for j in range(1, n+1):
            if p[j-1] in [s[i-1], '?']:
                dp[i][j] = dp[i-1][j-1]
            elif p[j-1] == '*':
                dp[i][j] = dp[i][j-1] or dp[i-1][j]
    return dp[m][n]
string, pat = inp.readline().strip().split(',')
string, pat = string[1:-1], pat[1:-1]
print(isMatch(string, pat))

True


In [8]:
import sys, collections
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
T = int(inp.readline().strip())
inf = 1<<31
for t in range(T):
    M, N = map(int, inp.readline().strip().split(' '))
    mat = []
    for _ in range(M):
        mat.append(list(inp.readline().strip()))
    if M == 0 or N == 0: print(-1)
    else:
        def bfs(x, y):
            Q = collections.deque([(x, y)])
            vis = [[inf]*N for _ in range(M)]
            vis[x][y] = 0
            ans = inf
            while Q:
#                 print(Q)
                i, j = Q.popleft()
                if i == 0 or i == M-1 or j==0 or j==N-1: ans = min(ans, vis[i][j])
                for di, dj in [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]:
                    if 0<=di<M and 0<=dj<N and mat[di][dj] not in ['@','#'] and vis[di][dj]>vis[i][j]+int(mat[di][dj]=='*'):
                        vis[di][dj] = vis[i][j]+int(mat[di][dj]=='*')
                        Q.append((di, dj))
            return ans if ans < inf else -1
        x, y = -1, -1
        for i in range(M):
            for j in range(N):
                if mat[i][j] == '@': 
                    x, y = i, j
                    break
        if x == -1: print(-1)
        elif x==0 or x == M-1 or y==0 or y==N-1: print(0)
        else:
#             print(x, y)
            print(bfs(x, y))

1
0
-1


In [9]:
import sys, collections
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
N,Q = map(int,inp.readline().strip().split(' '))
parent = [-1]
for val in inp.readline().strip().split(' '):
    parent.append(int(val)-1)
class TreeAncestor(object):
    def __init__(self, n, parent):
        self.dp = [[] for _ in range(n)]
        for i in range(n):
            self.dp[i].append(parent[i])
        j = 1
        while 1:
            allneg = 1
            for i in range(n):
                nxt = self.dp[self.dp[i][j-1]][j-1] if self.dp[i][j-1]!=-1 else -1
                self.dp[i].append(nxt)
                if nxt!=-1: allneg = 0
            if allneg: break
            j+=1


    def getKthAncestor(self, node, k):
        if k == 0 or node == -1: return node
        pos, ans = 0, node
        while k and ans!=-1:
            if pos >= len(self.dp[ans]): return -1
            if k&1: ans = self.dp[ans][pos]
            pos += 1
            k >>= 1
        return ans
tree = TreeAncestor(N, parent)
for _ in range(Q):
    x, k = map(int,inp.readline().strip().split(' '))
    print(tree.getKthAncestor(x-1, k)+1)

4
0
2


In [7]:
import sys, collections, math, bisect
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
def preprocess(N):
    ans = [0] * N
    ans[0] = ans[1] = 1
    for i in range(2, int(math.sqrt(N))):
        for j in range(i*i,N,i):
            ans[j] = 1
    return [i for i in range(2, N) if ans[i]==0]
table = preprocess(1000000)
while True:
    n = int(inp.readline().strip())
    res = []
    if n == 2: res.append(n)
    else:
        idx = len(table)-1
        while n > 1:
            idx = bisect.bisect_left(table, n)
            if n%table[idx]==0:
                res.append(table[idx])
                n = n/table[idx]
            else:
                while idx>=0 and n%table[idx]!=0:
                    idx -= 1
                res.append(table[idx])
                n = n / table[idx]
    print(res[::-1])

[2, 3]
[2, 2, 2]
[2, 5]
[11]
[3, 7]
[19]
[89]
[2, 11]


ValueError: invalid literal for int() with base 10: ''

In [1]:
import sys, collections, math, bisect
FILE = 1
inf = 1<<31
inp = sys.stdin if not FILE else open('input.txt', 'r')
M, N = map(int,inp.readline().strip().split(' '))
mat = []
for _ in range(M):
    mat.append(list(inp.readline().strip()))
si, sj = -1, -1
for i in range(M):
    for j in range(N):
        if mat[i][j] == 'T':
            si,sj = i, j
            break
Q = collections.deque([(si,sj,0)])
ans = inf
res = []
while Q:
    x, y, cost = Q.popleft()
    if cost > ans: break
    if mat[x][y]=='X': 
        ans = min(ans, cost)
        res.append((x,y))
    mat[x][y] = '1'
    for dx,dy in [(x-1,y),(x+1,y),(x,y-1),(x,y+1)]:
        if 0<=dx<M and 0<=dy<N and mat[dx][dy]!='1':
            Q.append((dx,dy,cost+1))
res.sort()
print(ans)
if ans > 0:
    print(' '.join([str(a)+' '+str(b) for a, b in res]))

4
0 0 1 5


In [2]:
import sys, collections, math, bisect
FILE = 1
inf = 1<<31
inp = sys.stdin if not FILE else open('input.txt', 'r')
dd = {(0,4,4,4,4,4,4,4,0,0):0,(0,2,3,2,2,2,2,4,0,0):1,(0,4,6,2,2,2,2,6,0,0):2,
      (0,4,4,2,4,2,4,4,0,0):3,(0,2,3,4,4,7,2,2,0,0):4,(0,2,6,2,5,2,4,4,0,0):5,
      (0,4,2,2,5,4,4,4,0,0):6,(0,6,6,2,2,2,2,2,0,0):7,(0,4,4,4,5,2,2,4,0,0):9}
T = int(inp.readline().strip())
for _ in range(T):
    N = int(inp.readline().strip())
    D = N/10
    mat = []
    for i in range(N):
        mat.append(map(int,inp.readline().strip().split(' ')))
    p = []
    for i in range(10):
        p.append(sum(mat[i*D])/D)
    res = dd[tuple(p)]
    if res !=0: print(res)
    else:
        if mat[4*D][4*D]==0: print(0)
        else:print(8)

9
4


In [ ]:
import sys, collections, math, bisect
FILE = 1
inf = 1<<31
inp = sys.stdin if not FILE else open('input.txt', 'r')
T = int(inp.readline().strip())
for _ in range(T):
    N, K = map(int,inp.readline().strip().split(' '))
    arr = map(int,inp.readline().strip().split(' '))
    edges = set()
    for _ in range(N-1):
        a, b = map(int, inp.readline().strip().split(' '))
        edges.add((a,b))
        edges.add((b,a))
    

In [7]:
import sys, collections, math, bisect
FILE = 1
inf = 1<<31
inp = sys.stdin if not FILE else open('input.txt', 'r')
mod = 10**9+7
N, M = map(int,inp.readline().strip().split(' '))
if M == 1: print(1)
elif M == 2: 
    if N <= 2: print(N)
    else:
        a, b = 1, 2
        for i in range(3, N+1):
            c = (a+b)%mod
            a = b
            b = c
        print(c)
elif M == 3:
    if N <= 2: print(1 if N == 1 else 3)
    a, b = 1, 3
    for i in range(3, N+1):
        c = (a*2+b)%mod
        a = b
        b = c
    print(c)
elif M == 4:
    dp = [0] * (N+1)
    bp = [0] * (N+1)
    dp[1], dp[2] = 1, 5
    bp[1], bp[2] = 1, 2
    for i in range(3, N+1):
        bp[i] = (dp[i-1]+bp[i-1])%mod
        dp[i] = (dp[i-1]+3*dp[i-2]+2*bp[i-2])%mod
    print(dp[N])
else:
    dp = [0] * (N+1)
    bp = [0] * (N+1)
    dp[1], dp[2] = 1, 8
    for i in range(3, N+1):
        dp[i] = (dp[i-1]+7*dp[i-2])

5


In [4]:
import sys, collections, math, bisect
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
T = int(inp.readline().strip())
for t in range(T):
    N, M = map(int,inp.readline().strip().split(' '))
    mat = [[0]*N for _ in range(N)]
    if N%2: mat[N/2][N/2] = N*N
    cnt = 0
    for c in range(N/2):
        si, sj, d, w = c, c, c%2, N-c*2
        row, col = si, sj
        if d==0:
            while col < sj+w:
                cnt += 1
                mat[row][col] = cnt
                col += 1
            row, col = si+1, sj+w-1
            while row < si+w:
                cnt += 1
                mat[row][col] = cnt
                row += 1
            row, col = si+w-1, sj+w-2
            while col>=sj:
                cnt += 1
                mat[row][col] = cnt
                col -= 1
            row, col = si+w-2, sj
            while row>si:
                cnt += 1
                mat[row][col] = cnt
                row -= 1
        else:
            while row<si+w:
                cnt += 1
                mat[row][col] = cnt
                row += 1
            row, col = si+w-1, sj+1
            while col < sj+w:
                cnt += 1
                mat[row][col] = cnt
                col += 1
            row, col = si+w-2, sj+w-1
            while row >= si:
                cnt += 1
                mat[row][col] = cnt
                row -= 1
            row, col = si, sj+w-2
            while col > sj:
                cnt += 1
                mat[row][col] = cnt
                col -= 1
    print('\n'.join([' '.join(map(str,x)) for x in mat]))
    for _ in range(M):
        i, j = map(int,inp.readline().strip().split(' '))
        print(mat[i][j])

1 2 3 4 5 6
20 21 32 31 30 7
19 22 33 34 29 8
18 23 36 35 28 9
17 24 25 26 27 10
16 15 14 13 12 11
3
19
27


In [15]:
import sys, collections, math, bisect
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
class FileSet(object):
    def __init__(self):
        self.a = {}
        self.b = {}
        self.id = 0
    def get(self, filename):
        fid = self.a.get(filename, self.id)
        if fid == self.id: 
            self.a[filename] = fid
            self.b[fid] = filename
            self.id += 1
        return fid
def bin_right(arr, idx, xlen):
    low, high = 0, xlen
    while low < high:
        mid = (low+high)>>1
        l, r = arr[mid]
        if l<=idx<=r: return mid
        if idx < l: high = mid
        elif idx > r: low = mid+1
    return low
class Arr(object):
    def __init__(self):
        self.arr = []
        self.len = 0
    def add(self, idx):
        if not self.len:
            self.arr.append([0,0])
            self.len += 1
            return
        loc = bin_right(self.arr, idx, self.len)
        if loc>=self.len:
            if self.arr[loc-1][1]==idx-1: self.arr[loc-1][1] = idx
            else: 
                self.arr.insert(loc,[idx,idx])
                self.len += 1
        elif loc==0:
            if self.arr[loc][0]==idx+1: self.arr[loc][0] = idx
            else:
                self.arr.insert(loc, [idx,idx])
                self.len += 1
        else:
            if self.arr[loc-1][1]==idx-1 and self.arr[loc][0]==idx+1:
                self.arr[loc-1][1] = self.arr[loc][1]
                del self.arr[loc]
                self.len -= 1
            elif self.arr[loc-1][1]==idx-1: self.arr[loc-1][1] = idx
            elif self.arr[loc][0] == idx+1: self.arr[loc][0] = idx
            else:
                self.arr.insert(loc, [idx,idx])
                self.len += 1
        
    def remove(self, idx):
        loc = bin_right(self.arr, idx, self.len)
        if self.arr[loc][0]==self.arr[loc][1]:
            del self.arr[loc]
            self.len -= 1
        elif idx == self.arr[loc][0]:
            self.arr[loc][0] = idx+1
        elif idx == self.arr[loc][1]:
            self.arr[loc][1] = idx-1
        else:
            l, r = self.arr[loc]
            self.arr[loc][1] = idx-1
            self.arr.insert(loc+1, [idx+1,r])
            self.len += 1
    
T = int(inp.readline().strip())
for t in range(T):
    N = int(inp.readline().strip())
    fs = FileSet()
    arrc = Arr()
    dd = {}
    for _ in range(N):
        param = inp.readline().strip().split(' ')
        if param[0] == 'open':
            if not arrc.len or arrc.arr[0][0]>0: idx = 0
            else: idx = arrc.arr[0][1]+1
            dd[idx] = fs.get(param[1])
            print(idx)
            arrc.add(idx)
        elif param[0] == 'dup':
            if not arrc.len or arrc.arr[0][0]>0: idx = 0
            else: idx = arrc.arr[0][1]+1
            dd[idx] = dd[int(param[1])]
            print(idx)
            arrc.add(idx)
        elif param[0] == 'dup2':
            p1, p2 = map(int, param[1:])
            if dd.get(p2, -1) == -1:
                dd[p2] = dd[p1]
                arrc.add(p2)
            else:
                dd[p2] = dd[p1]
        elif param[0] == 'close':
            p1 = int(param[1])
            dd[p1] = -1
            arrc.remove(p1)
        else:
            p1 = int(param[1])
            print(fs.b[dd[p1]])

0
1
2
3
libm.so
libc.so
libdl.so
0
0
0
0
1
output2.txt
1.txt


In [21]:
import sys, collections, math, bisect
import numpy as np
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
T = int(inp.readline().strip())
for t in range(T):
    N, D = map(int, inp.readline().strip().split(' '))
    data, ys = [], []
    for _ in range(N):
        arr = map(int, inp.readline().strip().split(' '))
        data.append(np.array(arr[:-1]))
        ys.append(arr[-1])
    M, K = map(int, inp.readline().strip().split(' '))
    nums = map(int, inp.readline().strip().split(' '))
    mats = []
    mats.append(np.zeros((D, nums[0]), dtype=int))
    if M==2: mats.append(np.zeros((nums[0],nums[1]), dtype=int))
    mats.append(np.array([0]*nums[-1]))
    dd = [0,0,0]
    edges = []
    for _ in range(K):
        arr = inp.readline().strip().split(' ')
        a, b, w = arr[0], arr[1], int(arr[2])
        if b == 'output':
            i, j = map(int, a.split('_')[1:])
            mats[-1][j-1] = w
            edges.append((-1, j-1,2))
            dd[2] += 1
        elif a.startswith('input'):
            k = int(a.split('_')[-1])
            i, j = map(int, b.split('_')[1:])
            mats[0][k-1][j-1] = w
            edges.append((0,k-1,j-1,0))
            dd[0] += 1
        else:
            i0,j0 = map(int, a.split('_')[1:])
            i1,j1 = map(int, b.split('_')[1:])
            mats[1][j0-1][j1-1] = w
            edges.append((1,j0-1,j1-1,1))
            dd[1] += 1
    def compute(mats, data, ys, nums):
        loss = 0
        for x, y in zip(data, ys):
            res = [np.dot(x, mats[0][:,i]) for i in range(nums[0])]
            if len(nums)==2:
                res = [np.dot(res, mats[1][:,i]) for i in range(nums[1])]
            loss += abs(np.dot(res, mats[-1])-y)
        return loss
    loss = compute(mats, data, ys, nums)
    inf = 1<<31
    losscur = inf
    for _, tup in enumerate(edges):
        if len(tup) == 4:
            i,j,k,kind = tup
            if kind==0 and dd[0] <= 1: continue
            if kind==1 and dd[1] <= 1: continue
            tmp = mats[i][j][k]
            mats[i][j][k] = 0
            losscur = min(losscur, compute(mats, data, ys, nums))
            mats[i][j][k] = tmp
        else:
            if dd[0] <= 1: continue
            i,j,kind = tup
            tmp = mats[i][j]
            mats[i][j] = 0
            losscur = min(losscur, compute(mats, data, ys, nums))
            mats[i][j] = tmp
    print(-1 if losscur>=loss else loss-losscur)

6
-1
180


In [ ]:
3
1 2
3 4 7
1 5
2
input_1 hidden_1_1 4
input_2 hidden_1_1 -2
input_2 hidden_1_2 1
hidden_1_1 output 2
hidden_1_2 output -2
2 2
3 4 0
4 3 17
1 5
2
input_1 hidden_1_1 4
input_2 hidden_1_1 -2
input_2 hidden_1_2 1
hidden_1_1 output 2
hidden_1_2 output -2
2 3
1 2 3 2
2 3 4 8
2 12
3 2
input_1 hidden_1_1 -3
input_2 hidden_1_2 -1
input_3 hidden_1_3 2
input_1 hidden_1_2 -3
input_3 hidden_1_2 1
hidden_1_1 hidden_2_1 5
hidden_1_1 hidden_2_2 -5
hidden_1_2 hidden_2_1 -2
hidden_1_2 hidden_2_2 -1
hidden_1_3 hidden_2_1 -2
hidden_2_1 output 2
hidden_2_2 output -2

In [18]:
# Compute AUROC
def auroc(y_list, s_list):
    idxs = np.argsort(y_list)[::-1]
    y_list = y_list[idxs]
    s_list = s_list[idxs]
    M = len(s_list)
    P = list(s_list).count(1)
    N = M - P
    auc, curh = 0., 0.
    for i in range(M):
        if s_list[i] == 1: curh += 1./P
        else: auc += curh / N
    return auc

In [19]:
auroc(pred, y)

0.75

In [20]:
from sklearn import metrics
import numpy as np
y = np.array([0, 0, 1, 1])
pred = np.array([0.1, 0.4, 0.35, 0.8])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.75

In [16]:
import sys, collections, math, bisect
import numpy as np
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
n = list(inp.readline().strip())
def next_n(string, idx):
    add = 0
    for j in range(idx, -1, -1):
        string[idx] = int(string[idx])+1
        add += string[idx]/10
        string[idx] = str(string[idx]%10)
        if add == 0: break
    return string[:idx+1] + ['0']*(len(string)-idx-1)
while 1:
    idx = -1
    for i in range(len(n)-1, 0, -1):
        if n[i]>n[i-1]:
            idx = i
            break
    if idx == -1: break
#     print(n)
    n = next_n(n, idx-1)
print(int(''.join(n)))

11


In [1]:
import sys, collections, math, bisect
import numpy as np
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
line = inp.readline().strip()[1:-1]
def lowbit(x): return x&(-x)
def insert(x, tmp, n):
    i = x
    while i <= n:
        if dd[i] < tmp: dd[i] = tmp
        i += lowbit(i)
def getsum(x):
    ret = -1
    i = x
    while i>=1:
        if dd[i] > ret: ret = dd[i]
        i -= lowbit(i)
    return ret
def clear(x, n):
    i = x
    while i<=n:
        dd[i] = 0
        i += lowbit(i)
def solve(l, r):
    if l==r: return
    mid = (l+r)>>1
    solve(l, mid)
    for i in range(mid+1, r+1):
        now[i] = arr[i]
    arr.sort
import sys, collections, math, bisect
import numpy as np
FILE = 0
inp = sys.stdin if not FILE else open('input.txt', 'r')
line = inp.readline().strip()[1:-1]
if len(line) == 0: print(0)
else:
    line += ','
    arr = []
    for tmp in line.split('],')[:-1]:
        arr.append(tuple(map(int, tmp[1:].split(','))))
    ans = [(min(x),max(x)) for x in arr]
    

[[2, 3, 3], [3, 3, 4]]


In [5]:
import sys, collections, math, bisect
FILE = 0
inp = sys.stdin if not FILE else open('input.txt', 'r')
T = int(inp.readline().strip())
for t in range(T):
    N = int(inp.readline().strip())
    arr = list(inp.readline().strip())
    print(arr)
    arr = ['0'] + arr + ['0']
    left, right = [0] * (N+2), [0] * (N+2)
    for i in range(1, N+2):
        if arr[i-1] == '1': left[i] = left[i-1]+1
    for i in range(N, -1, -1):
        if arr[i+1] == '1': right[i] = right[i+1]+1
    ans = 0
    for i in range(N+2):
        ans = max(ans, left[i]+right[i])
    print(ans)

['1', '0', '1', '0', '0', '1', '1']
2
['1', '0', '1', '1', '1', '0', '0']
4


In [11]:
# py3
import sys, collections, math, bisect
FILE = 0
inp = sys.stdin if not FILE else open('input.txt', 'r')
N = int(inp.readline().strip())
inf = 1<<30
dist = [[inf]*N for _ in range(N)]
for i in range(N-1):
    a, b, c = list(map(int, inp.readline().strip().split(' ')))
    dist[a-1][b-1] = dist[b-1][a-1] = c
for i in range(N):
    dist[i][i] = 0
for k in range(N):
    for i in range(N):
        for j in range(N):
            dist[i][j] = min(dist[i][j], dist[i][k]+dist[k][j])
tmp = list(map(int, inp.readline().strip().split(' ')))
k1, l1 = tmp[0], tmp[1:]
tmp = list(map(int, inp.readline().strip().split(' ')))
k2, l2 = tmp[0], tmp[1:]
tmp = list(map(int, inp.readline().strip().split(' ')))
k3, l3 = tmp[0], tmp[1:]
def compute(dist, i, j, k):
    d = 1<<30
    for t in range(N):
        d = min(d, dist[t][i]+dist[t][j]+dist[t][k])
    return d
ans = 0
for i in l1:
    for j in l2:
        for k in l3:
            ans += compute(dist, i-1, j-1, k-1)
return ans/k1/k2/k3

13.5

In [7]:
1<<30

1073741824

In [5]:
# py3
import sys, collections, math, bisect
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
N = int(inp.readline().strip())
keydict = {}
for i, ch in enumerate('sdupf0%'):
    if ch!='0': keydict[ch] = i+3
    else: 
        for z in '0123456789':
            keydict[z] = i+3
print(keydict)
for _ in range(N):
    string = inp.readline().strip()
    num_p = string.count('%')
    ans = 0
    if not num_p: ans = len(string)*3+1
    else:
        token = 0
        for ch in string:
            if token == 0:
                ans += 3
                if ch == '%': token = 1
                continue
            else:
                ans += keydict.get(ch, keydict['%'])
                if ch == '%': token = 0
        ans += 1
    print(ans)

{'s': 3, 'd': 4, 'u': 5, 'p': 6, 'f': 7, '0': 8, '1': 8, '2': 8, '3': 8, '4': 8, '5': 8, '6': 8, '7': 8, '8': 8, '9': 8, '%': 9}
16
19
22
52
62
28


In [6]:
8+8+8+6+9+9

48

In [13]:
# py3
import sys, collections, math, bisect
FILE = 1
inp = sys.stdin if not FILE else open('input.txt', 'r')
T = int(inp.readline().strip())
for _ in range(T):
    M, N, X, Y, P = map(int, inp.readline().strip().split(' '))
    Z = M*X-N*Y
    if Z <=0 and P-100000-M*X>0: print(-1)
    else:
        B = P-100000-M*X
        if B <=0:
            Q = int(math.ceil((P-100000)*1./X))
        else:
            day = int(math.ceil(B*1./Z))
            Q = day*(M+N)+int(math.ceil((P-100000-day*Z)*1./X))
        print(Q)

3
-1
2


In [6]:
from random import randint
N = 5

def rand_permutation(N):
	arr = list(range(1,N+1))
	for i in range(1, N):
		j = randint(0, i)
		tmp = arr[i]
		arr[i] = arr[j]
		arr[j] = tmp
	return arr

answer = rand_permutation(N)

print(' '.join([str(x) for x in answer]))

4 5 2 1 3


In [2]:
ans = {}
class Node():
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None
        
def right_scene(root, height=0):
    if not root: return
    if height not in ans: ans[height] = root.val
    right_scene(root.right, height+1)
    right_scene(root.left, height+1)

##
##     0
##   1    2
## 3   4 5
##6  7
treeA = Node(0)
treeA.left = Node(1)
treeA.right = Node(2)
treeA.left.left = Node(3)
treeA.left.right = Node(4)
treeA.right.left = Node(5)
treeA.left.left.left = Node(6)
treeA.left.left.right = Node(7)

right_scene(treeA, 0)
print(ans.values())

dict_values([0, 2, 5, 7])
